In [218]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import date, timedelta
import time

In [13]:
path_to_chromedriver = '/Users/skylershi/Data Science/chromedriver' # Path to access a chrome driver
browser = webdriver.Chrome(executable_path=path_to_chromedriver)

url = 'https://stats.nba.com/scores'
browser.get(url)

## Specify Scraping Date Range

In [3]:
start_date = date(2019, 12, 1)
end_date   = date(2020, 1, 1)
delta      = timedelta(days=1)

The endpoint **stats.nba.com/scores** accepts dates in format**stats.nba.com/scores/{month}/{day}/{year}** where month and day are both zero-padded if necessary

In [10]:
date_scrape = start_date
while date_scrape <= end_date:
    date_str = date_scrape.strftime("%m/%d/%Y")
    date_scrape += delta
    
    

## Scrape Games Off Base Page

In [108]:
url = 'https://stats.nba.com/scores/12/07/2019'
browser.get(url)

In [110]:
browser.find_elements_by_partial_link_text('Box Score')

[<selenium.webdriver.remote.webelement.WebElement (session="d170af94a5571d2f7689ee0c629aede0", element="724a4893-9ba2-436f-a033-ea19dd6f449f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d170af94a5571d2f7689ee0c629aede0", element="a7a5d774-3628-4333-99a9-cbb4097eeb61")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d170af94a5571d2f7689ee0c629aede0", element="69a67e1e-59de-4dc7-810f-f15a2b10402d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d170af94a5571d2f7689ee0c629aede0", element="5c37f24b-58a3-4d23-8bbb-8fdda2c8fa84")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d170af94a5571d2f7689ee0c629aede0", element="b96d6b2f-a1c8-42a1-8d86-28b9da394500")>]

In [45]:
games = browser.find_elements_by_xpath('/html/body/main/div[2]/div/div[2]/div/div[2]/div[1]/div/div/*')

#xpath indexing starts at 1!
for game_id in range(3, len(games)+1):
    game_box_score_xpath = '''
                                /html/body/main/div[2]/div/div[2]/div/div[2]/div[1]/div/div/
                                div[{}]/div[2]/div[1]/div/div[2]/div/a[1]
                           '''.format(game_id)
    
    browser.find_element_by_xpath(game_box_score_xpath).click()
    time.sleep(1)
    
    # scrape boxscores
    
    
    browser.back()
    time.sleep(1)


## Scrape Boxscore off Game Page

In [198]:
url = 'https://stats.nba.com/game/0021900001/'
browser.get(url)

In [172]:
table = browser.find_element_by_xpath('/html/body/main/div[2]/div/div/div[4]/div/div[2]/div/nba-stat-table[1]')
table_text = table.text.replace('SCREEN\nASSISTS', 'SCREEN_ASSISTS')
for line_id, line in enumerate(table_text.split('\n')):
    print(line_id, '\t', line)
    if(line_id > 8):
        break

0 	 PLAYER MIN SCREEN ASSISTS SCREEN_ASSISTS PTS DEFLECTIONS OFF LOOSE BALLS
1 	 RECOVERED DEF LOOSE BALLS
2 	 RECOVERED LOOSE BALLS
3 	 RECOVERED CHARGES
4 	 DRAWN CONTESTED
5 	 2PT SHOTS CONTESTED
6 	 3PT SHOTS CONTESTED
7 	 SHOTS OFF BOX OUTS DEF BOX OUTS BOX
8 	 OUTS
9 	 Jrue Holiday F


In [194]:
table = browser.find_element_by_xpath('/html/body/main/div[2]/div/div/div[4]/div/div[2]/div/nba-stat-table[1]')

column_names = []
player_stats = []
temp_player_stat = []

text_replacements = {
    # Hustle stats replacement strings
    'SCREEN\nASSISTS': 'SCREEN_ASSISTS',
    'SCREEN ASSISTS' : 'SCREEN_ASSISTS',
    'SCREEN_ASSISTS PTS': 'SCREEN_ASSISTS_PTS',
    'OFF LOOSE BALLS\nRECOVERED': 'OFF_LOOSE_BALLS_RECOVERED',
    'DEF LOOSE BALLS\nRECOVERED': 'DEF_LOOSE_BALLS_RECOVERED',
    'LOOSE BALLS\nRECOVERED': 'LOOSE_BALLS_RECOVERED',
    'CHARGES\nDRAWN': 'CHARGES_DRAWN',
    'CONTESTED\n2PT SHOTS': 'CONTESTED_2PT_SHOTS',
    'CONTESTED\n3PT SHOTS': 'CONTESTED_3PT_SHOTS',
    'CONTESTED\nSHOTS': 'CONTESTED_SHOTS',
    'OFF BOX OUTS' : 'OFF_BOX_OUTS',
    'DEF BOX OUTS' : 'DEF_BOX_OUTS',
    'BOX\nOUTS': 'BOX_OUTS',
    # Advanced stats replacement strings
    ' RATIO': '_RATIO',
    # Defense stats replacement strings
    'DEF\nMIN': 'DEF_MIN',
    'PARTIAL\nPOSS': 'PARTIAL_POSS'
}

table_text = table.text
for orig_str, replace_str in text_replacements.items():
    table_text = table_text.replace(orig_str, replace_str)


for line_id, line in enumerate(table_text.split('\n')):
    if line_id == 0:
        column_names = line.split(' ')[1:]
        column_names.insert(0,'PLAYER')
    else:
        
        # stop reading once we see totals or DNP line
        if ('Totals' in line) or ('DNP' in line) or ('DND' in line):
            break

        if line_id % 2 == 1:
            line_cleaned = line[:-2] + line[-2:].replace(' F', '').replace(' C', '').replace(' G', '')
            temp_player_stat.append(line_cleaned)
        if line_id % 2 == 0:
            temp_player_stat.extend(line.split(' '))
            player_stats.append(temp_player_stat)
            temp_player_stat = []

In [195]:
df = pd.DataFrame(player_stats, columns = column_names)
df.head()

,PLAYER,MIN,OFFRTG,DEFRTG,NETRTG,AST%,AST/TO,AST_RATIO,OREB%,DREB%,REB%,TO_RATIO,EFG%,TS%,USG%,PACE,PIE
0,Jrue Holiday,41:05,107.8,122.0,-14.2,20.7,1.20,23.1,5.0,4.3,4.7,19.2,43.3,40.9,20.8,105.74,2.9
1,Brandon Ingram,35:06,102.6,122.5,-19.9,22.7,2.50,18.5,0.0,12.5,6.8,7.4,47.4,53.0,27.2,107.35,11.2
2,Derrick Favors,20:45,108.2,132.7,-24.5,11.8,2.00,22.2,4.8,26.1,15.9,11.1,50.0,50.0,13.7,113.31,5.6
3,JJ Redick,27:02,100.0,122.2,-22.2,5.6,0.33,7.7,0.0,6.7,3.4,23.1,88.9,88.9,17.9,111.86,8.0
4,Lonzo Ball,24:50,112.3,122.4,-10.1,22.7,5.00,35.7,0.0,17.2,9.6,7.1,42.9,50.8,15.3,111.14,8.0


## Scrape Multiple Boxscore views off of Game Page for Both Teams

In [214]:
url = 'https://stats.nba.com/game/0021900001/'
browser.get(url)

In [ ]:
/html/body/main/div[2]/div/div/div[4]/div/nav-dropdown/nav/section[2]/div/a

In [ ]:
/html/body/main/div[2]/div/div/div[4]/div/nav-dropdown/nav/section[2]/ul/li[1]/a

In [ ]:
/html/body/main/div[2]/div/div/div[4]/div/nav-dropdown/nav/section[2]/ul/li[1]/a

In [216]:
text_replacements = {
    # Hustle stats replacement strings
    'SCREEN\nASSISTS': 'SCREEN_ASSISTS',
    'SCREEN ASSISTS' : 'SCREEN_ASSISTS',
    'SCREEN_ASSISTS PTS': 'SCREEN_ASSISTS_PTS',
    'OFF LOOSE BALLS\nRECOVERED': 'OFF_LOOSE_BALLS_RECOVERED',
    'DEF LOOSE BALLS\nRECOVERED': 'DEF_LOOSE_BALLS_RECOVERED',
    'LOOSE BALLS\nRECOVERED': 'LOOSE_BALLS_RECOVERED',
    'CHARGES\nDRAWN': 'CHARGES_DRAWN',
    'CONTESTED\n2PT SHOTS': 'CONTESTED_2PT_SHOTS',
    'CONTESTED\n3PT SHOTS': 'CONTESTED_3PT_SHOTS',
    'CONTESTED\nSHOTS': 'CONTESTED_SHOTS',
    'OFF BOX OUTS' : 'OFF_BOX_OUTS',
    'DEF BOX OUTS' : 'DEF_BOX_OUTS',
    'BOX\nOUTS': 'BOX_OUTS',
    # Advanced stats replacement strings
    ' RATIO': '_RATIO',
    # Defense stats replacement strings
    'DEF\nMIN': 'DEF_MIN',
    'PARTIAL\nPOSS': 'PARTIAL_POSS'
}

In [123]:
options = ['Traditional', 'Advanced', 'Scoring', 'Usage', 'Player Tracking', 'Hustle', 'Defense']

In [219]:
# keep track of all stats with a list of dfs
team1_stats_dfs = []
team2_stats_dfs = []
team1 = browser.find_elements_by_class_name('nba-stat-table__caption')[0].text
team2 = browser.find_elements_by_class_name('nba-stat-table__caption')[1].text

# scrape multiple options designated above
for i, option in enumerate(options):
    # only select new options item if not first-time loading page
    if i != 0:
        browser.find_element_by_partial_link_text(option).click()
        time.sleep(2)


    # scrape 2 boxscore tables
    table1 = browser.find_element_by_xpath('/html/body/main/div[2]/div/div/div[4]/div/div[2]/div/nba-stat-table[1]')
    table2 = browser.find_element_by_xpath('/html/body/main/div[2]/div/div/div[4]/div/div[2]/div/nba-stat-table[2]')


    

    for table_idx, table in enumerate([table1, table2]):
        column_names = []
        player_stats = []
        temp_player_stat = []
        # replace all the unexpected spaces and newlines strings by our script
        table_text = table.text
        for orig_str, replace_str in text_replacements.items():
            table_text = table_text.replace(orig_str, replace_str)

        # read table text into python list of lists
        for line_id, line in enumerate(table_text.split('\n')):
            if line_id == 0:
                column_names = line.split(' ')[1:]
                column_names.insert(0,'PLAYER')
            else:
                # stop reading once we see totals or DNP/DND line
                if ('Totals' in line) or ('DNP' in line) or ('DND' in line):
                    break

                if line_id % 2 == 1:
                    line_cleaned = line[:-2] + line[-2:].replace(' F', '').replace(' C', '').replace(' G', '')
                    temp_player_stat.append(line_cleaned)
                if line_id % 2 == 0:
                    temp_player_stat.extend(line.split(' '))
                    player_stats.append(temp_player_stat)
                    temp_player_stat = []

        # convert list of lists into pandas df
        df = pd.DataFrame(player_stats, columns = column_names)
        if table_idx == 0:
            team1_stats_dfs.append(df)
        else:
            team2_stats_dfs.append(df)

    # scroll back to top of page
    browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)
    # open the options menu
    browser.find_element_by_partial_link_text(option).click()
    time.sleep(2)

In [226]:
browser.back()

## Putting All the Scraping Altogether

In [244]:
url = 'https://stats.nba.com/scores/12/07/2019'
browser.get(url)

In [245]:
# these are replacement strings for unexpected spaces and newlines by our script
text_replacements = {
    # Hustle stats replacement strings
    'SCREEN\nASSISTS': 'SCREEN_ASSISTS',
    'SCREEN ASSISTS' : 'SCREEN_ASSISTS',
    'SCREEN_ASSISTS PTS': 'SCREEN_ASSISTS_PTS',
    'OFF LOOSE BALLS\nRECOVERED': 'OFF_LOOSE_BALLS_RECOVERED',
    'DEF LOOSE BALLS\nRECOVERED': 'DEF_LOOSE_BALLS_RECOVERED',
    'LOOSE BALLS\nRECOVERED': 'LOOSE_BALLS_RECOVERED',
    'CHARGES\nDRAWN': 'CHARGES_DRAWN',
    'CONTESTED\n2PT SHOTS': 'CONTESTED_2PT_SHOTS',
    'CONTESTED\n3PT SHOTS': 'CONTESTED_3PT_SHOTS',
    'CONTESTED\nSHOTS': 'CONTESTED_SHOTS',
    'OFF BOX OUTS' : 'OFF_BOX_OUTS',
    'DEF BOX OUTS' : 'DEF_BOX_OUTS',
    'BOX\nOUTS': 'BOX_OUTS',
    # Advanced stats replacement strings
    ' RATIO': '_RATIO',
    # Defense stats replacement strings
    'DEF\nMIN': 'DEF_MIN',
    'PARTIAL\nPOSS': 'PARTIAL_POSS'
}

In [246]:
# all the options we want to scrape for
options = ['Traditional', 'Advanced', 'Usage', 'Player Tracking', 'Hustle', 'Defense']

In [247]:
# get all boxscore links
box_score_links = browser.find_elements_by_partial_link_text('Box Score')

for i in range(4,len(box_score_links)):
    
    browser.find_elements_by_partial_link_text('Box Score')[i].click()
    time.sleep(2)
    
    # browser is within boxscore page now
    
    # keep track of all stats with a list of dfs
    team1_stats_dfs = []
    team2_stats_dfs = []
    team1 = browser.find_elements_by_class_name('nba-stat-table__caption')[0].text
    team2 = browser.find_elements_by_class_name('nba-stat-table__caption')[1].text
    
    # scrape multiple options designated above
    for i, option in enumerate(options):
        # only select new options item if not first-time loading page
        if i != 0:
            browser.find_element_by_partial_link_text(option).click()
            time.sleep(2)

        
        # scrape 2 boxscore tables
        table1 = browser.find_element_by_xpath('/html/body/main/div[2]/div/div/div[4]/div/div[2]/div/nba-stat-table[1]')
        table2 = browser.find_element_by_xpath('/html/body/main/div[2]/div/div/div[4]/div/div[2]/div/nba-stat-table[2]')
        
        
        
        # scrape tables for both teams
        for table_idx, table in enumerate([table1, table2]):
            column_names = []
            player_stats = []
            temp_player_stat = []
            
            # replace all the unexpected spaces and newlines strings by our script
            table_text = table.text
            for orig_str, replace_str in text_replacements.items():
                table_text = table_text.replace(orig_str, replace_str)

            # read table text into python list of lists
            for line_id, line in enumerate(table_text.split('\n')):
                if line_id == 0:
                    column_names = line.split(' ')[1:]
                    column_names.insert(0,'PLAYER')
                else:
                    # stop reading once we see totals or DNP/DND line
                    if ('Totals' in line) or ('DNP' in line) or ('DND' in line):
                        break

                    if line_id % 2 == 1:
                        line_cleaned = line[:-2] + line[-2:].replace(' F', '').replace(' C', '').replace(' G', '')
                        temp_player_stat.append(line_cleaned)
                    if line_id % 2 == 0:
                        temp_player_stat.extend(line.split(' '))
                        player_stats.append(temp_player_stat)
                        temp_player_stat = []

            # convert list of lists into pandas df
            df = pd.DataFrame(player_stats, columns = column_names)
            if table_idx == 0:
                team1_stats_dfs.append(df)
            else:
                team2_stats_dfs.append(df)
        

        # open the options menu
        browser.find_element_by_partial_link_text(option).click()
        time.sleep(2)
        
    # combine all the stats and write to a pickle file
    
    
    # finish scraping the stats from all the options, go back to the previous page listing all games in one day
    browser.get(url)
    time.sleep(3)